In [3]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns
from scipy.stats import pearsonr

from zipfile import ZipFile
import scipy as sp
import re


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap


from scipy.special import binom

In [4]:
log_file_path = "../../mascot_glm/results/glm_random_3000_downsampled.log"


In [5]:
def read_in_Ne_changes_mascot(log_file_path):  
    
    Ne_skyline_dict = {"sample":[]}

    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if ("NeGLM."  in col) or ("migrationGLM." in col):
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        deme = name.split(".")[1]# the syntax here is "Ne.region.1" where region is deme and 1 is interval 1
                        #interval = name.split(".")[2]

                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])

    return(Ne_skyline_dict)

In [6]:
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

In [7]:
Ne_skyline

{'sample': ['0',
  '100000',
  '200000',
  '300000',
  '400000',
  '500000',
  '600000',
  '700000',
  '800000',
  '900000',
  '1000000',
  '1100000',
  '1200000',
  '1300000',
  '1400000',
  '1500000',
  '1600000',
  '1700000',
  '1800000',
  '1900000',
  '2000000',
  '2100000',
  '2200000',
  '2300000',
  '2400000',
  '2500000',
  '2600000',
  '2700000',
  '2800000',
  '2900000',
  '3000000',
  '3100000',
  '3200000',
  '3300000',
  '3400000',
  '3500000',
  '3600000',
  '3700000',
  '3800000',
  '3900000',
  '4000000',
  '4100000',
  '4200000',
  '4300000',
  '4400000',
  '4500000',
  '4600000',
  '4700000',
  '4800000',
  '4900000',
  '5000000',
  '5100000',
  '5200000',
  '5300000',
  '5400000',
  '5500000',
  '5600000',
  '5700000',
  '5800000',
  '5900000',
  '6000000',
  '6100000',
  '6200000',
  '6300000',
  '6400000',
  '6500000',
  '6600000',
  '6700000',
  '6800000',
  '6900000',
  '7000000',
  '7100000',
  '7200000',
  '7300000',
  '7400000',
  '7500000',
  '7600000',
  '7

In [8]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        print(i)
        if ("NeGLM."in i) or ("migrationGLM" in i):
            if "Clock" in i: 
                deme = i.split(".")[1]
            else:
                deme = i.split(".")[2]
            #print(deme)
            #interval = i.split(".")[2]
            #print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = 10**mean_log
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = 10**lower_hpd_log_95
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = 10**upper_hpd_log_95
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = 10**lower_hpd_log_50
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = 10**upper_hpd_log_50
            
#             try:
#                 next_local_series = input_df["Ne"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"deme":deme, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95})
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [9]:
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))
Ne_df

444


sample         NeGLM.1Clock NeGLM.1scaler.Ne_hosp_orig  \
0           0                  1.0                        0.0   
1      100000   0.8287654601387997                        0.0   
2      200000   1.1435433364035075                        0.0   
3      300000   1.1783318661990927                        0.0   
4      400000    1.205791816748952                        0.0   
..        ...                  ...                        ...   
439  43900000  0.05962924340328993                        0.0   
440  44000000  0.04284541954290711                        0.0   
441  44100000  0.04784418965432981                        0.0   
442  44200000  0.04301002983792624                        0.0   
443  44300000  0.06262767113656567                        0.0   

    NeGLM.1scaler.Ne_hosp_1wk_fwd NeGLM.1scaler.Ne_hosp_2wk_fwd  \
0                             0.0                           0.0   
1                             0.0            0.2937342636763922   
2             0.16547680331416847            0.3661636929360317   
3                             0.0            0.6880978393689812   
4                             0.0            0.6920924517361253   
..                            ...                           ...   
439            1.0565351196019965                           0.0   
440            1.2933499560364692                           0.0   
441            1.2109426641394507                           0.0   
442            1.1551808999276694                           0.0   
443            1.3288285506568251                           0.0   

    NeGLM.1scaler.percent_immunity NeGLM.1scaler.cases migrationGLM.1Clock  \
0                              0.0                 0.0                 1.0   
1                              0.0                 0.0  12.641876397693009   
2                              0.0                 0.0  12.049913279324743   
3                              0.0                 0.0  10.731662709270637   
4                              0.0                 0.0  13.053272056144033   
..                             ...                 ...                 ...   
439            -0.8503505466730279                 0.0   9.176728028116692   
440            -0.8375855780712445                 0.0   10.19582048013566   
441            -0.8375855780712445                 0.0   9.637629213341238   
442            -0.8375855780712445                 0.0  11.221272246599343   
443            -0.8072808206943587                 0.0   8.203627298724737   

    migrationGLM.1scaler.NPI_dates  \
0                              0.0   
1             -0.29206552838272315   
2                              0.0   
3                              0.0   
4                              0.0   
..                             ...   
439                            0.0   
440                            0.0   
441                            0.0   
442                            0.0   
443               1.48290606153211   

    migrationGLM.1scaler.safegraph_between_total_mvmt  
0                                                 0.0  
1                                  0.2166714862079584  
2                                                 0.0  
3                                                 0.0  
4                                                 0.0  
..                                                ...  
439                                               0.0  
440                                               0.0  
441                                               0.0  
442                                               0.0  
443                               0.28601644061746834  

[444 rows x 10 columns]

In [10]:
ne_summary = generate_summary_df(Ne_df)
ne_summary

sample
NeGLM.1Clock
NeGLM.1scaler.Ne_hosp_orig
NeGLM.1scaler.Ne_hosp_1wk_fwd
NeGLM.1scaler.Ne_hosp_2wk_fwd
NeGLM.1scaler.percent_immunity
NeGLM.1scaler.cases
migrationGLM.1Clock
migrationGLM.1scaler.NPI_dates
migrationGLM.1scaler.safegraph_between_total_mvmt


/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_82790/3821785173.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_82790/3821785173.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_82790/3821785173.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_82790/3821785173.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_

deme  mean_Ne_log  mean_Ne_linear  median_Ne_log  \
0                        1Clock     1.708374    5.109448e+01       1.293361   
0                  Ne_hosp_orig     0.114837    1.302678e+00       0.000000   
0               Ne_hosp_1wk_fwd     0.993802    9.858302e+00       1.138411   
0               Ne_hosp_2wk_fwd     0.096663    1.249291e+00       0.000000   
0              percent_immunity     0.099035    1.256131e+00       0.000000   
0                         cases     0.000000    1.000000e+00       0.000000   
0                        1Clock     9.803586    6.361894e+09       9.725285   
0                     NPI_dates     0.123847    1.329985e+00       0.000000   
0  safegraph_between_total_mvmt     0.016028    1.037596e+00       0.000000   

   upper_hpd_log_95  lower_hpd_log_95  upper_hpd_log_50  lower_hpd_log_50  \
0          4.629088          0.042845          1.293702          0.042845   
0          1.434234         -0.334592          0.000000          0.000000   
0          1.526271          0.000000          1.364246          0.990761   
0          0.842280         -0.219381          0.000000          0.000000   
0          1.195124         -0.850351          0.000000          0.000000   
0          0.000000          0.000000          0.000000          0.000000   
0         12.049913          7.660738         10.631582          9.087375   
0          1.149270         -0.604153          0.000000          0.000000   
0          0.286016         -0.121449          0.000000          0.000000   

   upper_hpd_linear  lower_hpd_linear  
0      4.256842e+04      1.103686e+00  
0      2.717903e+01      4.628154e-01  
0      3.359472e+01      1.000000e+00  
0      6.954727e+00      6.034185e-01  
0      1.567197e+01      1.411398e-01  
0      1.000000e+00      1.000000e+00  
0      1.121794e+12      4.578657e+07  
0      1.410164e+01      2.487978e-01  
0      1.932041e+00      7.560508e-01

In [15]:
month = {'month_predictor_0':'April 2022',
          'month_predictor_1':'May 2022',
         'month_predictor_2':'June 2022',
           'month_predictor_3': 'July 2022',
        'month_predictor_4':'August 2022', 
        'month_predictor_5':'September 2022',
         'month_predictor_6':'October 2022',
        'month_predictor_7':'November 2022',
        'month_predictor_8':'Decemeber 2022',
        'month_predictor_9':'Janaury 2023',
         'region_cases_weekly':'Weekly Cases',
         'region_cases_weekly_prevalence':'Weekly Prevalence',
         '_region_300Clock':'Clock', 
         "regions_air_travel_matrix":"Air Passenger Volumes"
}

In [16]:
ne_summary["predictor"] = ne_summary.deme.map(month)


In [33]:
ne_summary.to_csv("../data-files/predictor_coefficients.csv")

In [21]:
ne_pred = ne_summary.iloc[1:6]
mig_pred = ne_summary.iloc[-2:]

In [22]:
ne_pred

deme  mean_Ne_log  mean_Ne_linear  median_Ne_log  \
0      Ne_hosp_orig     0.114837        1.302678       0.000000   
0   Ne_hosp_1wk_fwd     0.993802        9.858302       1.138411   
0   Ne_hosp_2wk_fwd     0.096663        1.249291       0.000000   
0  percent_immunity     0.099035        1.256131       0.000000   
0             cases     0.000000        1.000000       0.000000   

   upper_hpd_log_95  lower_hpd_log_95  upper_hpd_log_50  lower_hpd_log_50  \
0          1.434234         -0.334592          0.000000          0.000000   
0          1.526271          0.000000          1.364246          0.990761   
0          0.842280         -0.219381          0.000000          0.000000   
0          1.195124         -0.850351          0.000000          0.000000   
0          0.000000          0.000000          0.000000          0.000000   

   upper_hpd_linear  lower_hpd_linear  
0         27.179030          0.462815  
0         33.594715          1.000000  
0          6.954727          0.603419  
0         15.671967          0.141140  
0          1.000000          1.000000

In [32]:
# error_bars = alt.Chart(mig_short).mark_errorbar(extent='ci').encode(
#   x=alt.X('mig_per_bl:Q', scale=alt.Scale(zero=False)),
#   y=alt.Y('migration_direction:N')
# )

points = alt.Chart(ne_pred).mark_point(filled=True, color='black', width = 5).encode(
  y=alt.Y('mean_Ne_log:Q',axis = alt.Axis(title = "Coefficient", grid =False, labelFontSize= 24,titleFontSize= 24) ),
  x=alt.X('deme', title = "Predictor",   axis=alt.Axis(labelFontSize= 24,titleFontSize= 24,labelLimit = 0, labelAngle=-35)),
).transform_filter(datum.predictor != "Clock" or datum.predictor != "air").properties(
    width=850,
    height=300
)

hpd = alt.Chart(ne_pred).mark_rule(strokeWidth = 2).encode(
    y2=alt.Y2("upper_hpd_log_95"),
    y = alt.Y("lower_hpd_log_95"),
    x=alt.X("deme")
).transform_filter(datum.predictor != "Clock").properties(
    width=850,
    height=300
)

one_line = alt.Chart(pd.DataFrame({'y': [0.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=850,
    height=300
)

predict_coeff = (points + hpd + one_line).properties(title='A')
predict_coeff.configure_view(
    strokeWidth=0
).configure_title(
    anchor='start', fontSize= 35
).configure_axis(
    labelFontSize=60,
    titleFontSize=60
).configure_legend(
    labelFontSize = 60)




alt.LayerChart(...)

In [31]:
# error_bars = alt.Chart(mig_short).mark_errorbar(extent='ci').encode(
#   x=alt.X('mig_per_bl:Q', scale=alt.Scale(zero=False)),
#   y=alt.Y('migration_direction:N')
# )

points = alt.Chart(mig_pred).mark_point(filled=True, color='black').encode(
  y=alt.Y('mean_Ne_log:Q',axis = alt.Axis(title = "Coefficient", grid =False, labelFontSize= 18,titleFontSize= 16) ),
  x=alt.X('deme', title = "Predictor",  axis=alt.Axis(labelFontSize= 16,titleFontSize= 16, labelAngle=-35)),
).transform_filter(datum.predictor != "Clock" ).properties(
    width=200,
    height=200
)

hpd = alt.Chart(mig_pred).mark_rule().encode(
    y2=alt.Y2("upper_hpd_log_95"),
    y = alt.Y("lower_hpd_log_95"),
    x=alt.X("deme")
).transform_filter(datum.predictor != "Clock").properties(
    width=300,
    height=150
)

one_line = alt.Chart(pd.DataFrame({'y': [0.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=300,
    height=150
)

predict_coeff = (points + hpd + one_line ).properties(title='B')
predict_coeff.configure_view(
    strokeWidth=0
).configure_title(
    anchor='start', fontSize= 25
).configure_axis(
    labelFontSize=24,
    titleFontSize=24
).configure_legend(
    labelFontSize = 60)




alt.LayerChart(...)